# Intro

We're going to Japan! One of the things that we're the most excited about for the trip is all of the great food places in Japan. We're certain that we'll be able to find good food almost anywhere but the sheer number of excellent restaurants in Japan is mindboggling. The most popular Japanese Restaurant rating site is Tabelog which is similar to Yelp. Unfortunately, their English version of the website does not allow you to search for the restaurants by name and you can only use filters to find a place. In order to make things easier to view, I thought it would be good to scrape the areas I was interested in for different uses. 

To help us orient ourselves, I wanted to have a map showing the top rated spots in the cities so we could keep an eye out while we wander. The goal of this is to put everything onto one spreadsheet and Google Maps so that I can have the flexibility of finding spots on the go or to filter out what I want to eat from the top rated restaurants. The current workflow is scraping tabelog.com for the ranking data $\rightarrow$ outputtings a .csv file $\rightarrow$ importing to Google Sheets $\rightarrow$ export the addresses to Google MyMaps. This will import everything onto a layer that can be viewed when using Google Maps and give us the ability to do some filtering and searching in Google Sheets while on the go.

#### Code
I will be using BeautifulSoup to scrape the webpages and pandas to organize and export the data. I'm able to set the amount of pages that I want to grab from the top rated categories as well as the region I'm interested in. If I wanted to, I could loop this process over a list of cities. However, this is not possible in reality as I would get rate limited and kicked off Tabelog for accessing too many pages. Thus, I utilized single runs for each city. It's possible that I could add in a delay after every page to prevent being kicked off if I wanted to try automating this for a list of regions.

In [5]:
# Importing libraries to use
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Set number of pages to get
max_pages = 0
page_count = 1
city = '' # city name like 'kyoto'. blank for not running when testing other bits

names = []
ratings = []
reviews = []
areas = []
genres = []
urls = []
lunch_price = []
dinner_price = []
addresses = []

# Base information
url = 'https://tabelog.com/en/' + city + '/rstLst/' + str(page_count) +'/?SrtT=rt'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

while page_count <= max_pages:

    print('On page ', page_count)
    # Extracting names, ratings, and areas from the soup object
    names_list = soup.find_all(class_='list-rst__rst-name-target cpy-rst-name')
    ratings_list = soup.find_all(class_='c-rating__val c-rating__val--strong list-rst__rating-val')
    reviews_list = soup.find_all(class_='list-rst__rvw-count-num cpy-review-count')
    area_list = soup.find_all(class_="list-rst__area-genre cpy-area-genre")
    soup_list = soup.find_all('div', class_='list-rst js-bookmark js-rst-cassette-wrap')

    # Extracting data and populating lists
    for i, (name, rating, area, review) in enumerate(zip(names_list, ratings_list, area_list, reviews_list)):
        names.append(name.get_text(strip=True))
        ratings.append(rating.get_text(strip=True))
        reviews.append(review.get_text(strip=True))

        # Extracting the URL for each restaurant on the page
        urls.append(soup_list[i].get('data-detail-url'))
        
        # Getting URL information for the address
        temp_page = requests.get(urls[-1])
        url_soup = BeautifulSoup(temp_page.text,'html.parser')
        addresses.append(url_soup.find(class_="rstinfo-table__address").get_text(strip=True)) 

        # Extracting lunch and dinner prices
        dinner_price.append(soup_list[i].find_all('span', class_='c-rating-v3__val')[0].get_text(strip=True))
        lunch_price.append(soup_list[i].find_all('span', class_='c-rating-v3__val')[1].get_text(strip=True)) 

        # Extracting area and genre. Some places don't show a nearest station so there isnt a / symbol to separate.
        if '/' in (soup_list[i].find('div', class_='list-rst__area-genre cpy-area-genre').get_text(strip=True)):
            area_genre = soup_list[i].find('div', class_='list-rst__area-genre cpy-area-genre').get_text(strip=True)
            temp_station = area_genre.split(' / ')[0].strip()
            temp_station = temp_station[:-5]
            areas.append(temp_station)
            genres.append(area_genre.split(' / ')[1].strip())
        else:
            areas.append('-')
            genres.append(soup_list[i].find('div', class_='list-rst__area-genre cpy-area-genre').get_text().replace('\n', ''))

    page_count += 1

    # Get new page of results to scrape and reload
    new_url = 'https://tabelog.com/en/'+city+'/rstLst/' + str(page_count) +'/?SrtT=rt'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.text,'html.parser')

# Creating a dictionary with the extracted data
data = {
    'Name': names,
    'Rating': ratings,
    'Reviews': reviews,
    'Station': areas,
    'Address': addresses,
    'Genre': genres,
    'Lunch Price': lunch_price,
    'Dinner Price': dinner_price,
    'URL': urls
}

# Creating a DataFrame from the data dictionary
df = pd.DataFrame(data)
df.to_csv(city+'-top'+str(max_pages)+'pages.csv')

Once it outputs, we get something like this if we import it to Google Sheets to view

<div>
<img src="img\sheets_preview.png" width="1024"/>
</div>

Success! Now that it works, I run the same script for a couple regions that we're interested in visiting as well as a general Top X of Japan where X is whatever number I want. The data can be formatted to look better and used for the trip

# Use case: Finding what is close by - Importing data into Google Maps


Now that the sheets can be prepared, I can now set up some Google Map layers to view the list of places I scraped while using Google Maps to navigate. In order to do this with the spreadsheet that I made, I need to use Google MyMaps instead of Google Maps. MyMaps is different than just Maps as it allows for the importing of data from external sources like .csvs, Google Sheets, etc. 

Once I've uploaded different spreadsheets that I made from scraping for the Top 1000 restaurants in Kyoto, Hokkaido, Tokyo, Osaka, and Japan onto MyMaps, I have something like this! Unfortunately, some of the addresses that were scraped were unable to be found when searching them up on Google Maps so they are not shown on the map
<div>
<img src="img\mymaps.png" width="512"/>
</div>




# Use case: Finding a food you want - Linking Google Sheets to Maps

Now that the maps integration is done, a little bit of work needs to be done to the data to make it usable under different situations. The current use case for the maps integration is to locate places and give us ideas when we are walking around specific areas looking for somewhere to eat. But what if we want to look for something specific like ice cream, ramen, or even curry? Filtering based on the genres generated from the scraped data should give us some the information.

When looking at the spreadsheet, the genres that were obtained from Tabelog may sometimes contain multiple tags. 
<div>
<img src="img\genres.png"/>
</div>

However, this is still searchable on mobile and on desktop versions of the sheets as we can type what we want to find, even if multiple tags are in the same cell. 
<div>
<img src="img\filter.png"/>
</div>
There is some room for improvement here to make things a little more streamlined by making the tags in different cells since they are separated with commas. This would make it so that tags are searchable by clicking on the filter rather than typing something out. Alternatively, I could make separate columns for each tag associated with the restaurants and filter for them across the selected columns. However, these aren't convenient when using the spreadsheet on a mobile device so the current method will suffice.

Now that I know what options are available for ice cream, I want to check out where it is. Luckily, Google Sheets have Smart Chips which allow a link that opens up that location on Google Maps. Unluckily, it only works with Google Maps links that are for a specific place or entering the address manually and selecting the correct option when creating the Smart Chip. Both of these can't be used without access to the developer APIs or by going through them individually unfortunately. However, a workaround that I thought about was creating a Google Maps link query based on the address that I scraped which can be clicked on to open up Google Maps. I could make it more specific as well by including the name as some locations have multiple listings. To create a link that is recognized by Sheets, I use the HYPERLINK and CONCATENATE functions on Google Sheets along with the url ("https://www.google.com/maps/search/?api=1&query=") to make the Google Maps search links found from the Google Maps API (https://developers.google.com/maps/documentation/urls/get-started). Since some of the restaurants have spaces in their names and this does not work super well when creating Google Maps links that are detectable by Google Sheets, a quick SUBSTITUTE function can do the job to change the white spaces to + signs which are accepted.

Full function used was =HYPERLINK(CONCATENATE("https://www.google.com/maps/search/?api=1&query=", SUBSTITUTE(B2," ","+"),"+", SUBSTITUTE(F2," ","+")), "Link")

Testing it for our first entry gives us this:

<div>
<img src="img\maps_link.png"/>
</div>

Which lets us click on something from the Sheets view to bring us to the Maps location!

# Formatting for usability

Now that we can use the sheet to look for places and open them up on Google Maps, I want to edit the spreadsheet for better readability. 


## Lunch and Dinner Price - Changing text to numbers and conditional formatting


The outputs from the lunch and dinner prices for each restaurant were taken as text ranges like JPY 40,000 ~ JPY 49,999. I want to modify these cells to make them searchable and easy to read
<div>
<img src="img\sheets_preview.png" width="1024"/>
</div>

Since the ranges given from Tabelog are all specific ranges, I can replace them with a substitute function and a multiple conditional using IFS() to check which range it is. As an example, I will say the range of JPY 40,000 ~ JPY 49,999 will be changed to the average of the two which is 45000 for a number that can be used for conditional formatting and sorting. In the case of the edge ranges of <999 JPY, >100,000 JPY, or '-' (did not exist in the scrape), numbers of 999 and 100,000, and a blank cell will be used respectively.


An example would be =IFS(I2 = "～JPY 999", 999 ,I2 = "JPY 1,000～JPY 1,999", 1500 ,I2 = "JPY 2,000～JPY 2,999", 2500,....) and keep listing them for all ranges known. I apply these in a new column where I can then use conditional formatting with a colour scale to give me a visual indicator of their prices. The initial price columns with ranges have been moved to the far right so that the formatted price columns are closer to the names and locations

<div>
<img src="img\conditional_price.png" width="1024"/>
</div>


# Final touch ups

Add in some extra formatting to help visibility with alternating row colours and reordered columns and it's complete for the Tokyo list! Repeat the process for other regions

<div>
<img src="img\final.png" width="1024"/>
</div>


# Testing

#### Important info for each restaurant nested in


All info per restaurant
```
<div class="list-rst__rst-data">
```
```
<div class="list-rst js-bookmark js-rst-cassette-wrap" data-detail-url="https://tabelog.com/en/tokyo/A1302/A130204/13018162/" data-rst-id="13018162">
```

Restaurant name
```
<a class="list-rst__rst-name-target cpy-rst-name" target="_blank" rel="noopener" data-list-dest="item_top" href="https://tabelog.com/en/tokyo/A1302/A130204/13018162/">Sugi ta</a>
```

Prices
```
<li class="c-rating-v3 list-rst__info-item">
                            <i aria-label="Average dinner price" class="c-rating-v3__time c-rating-v3__time--dinner" role="img"></i><span class="c-rating-v3__val">JPY 40,000～JPY 49,999</span>
                          </li>

<li class="c-rating-v3 list-rst__info-item">
                            <i aria-label="Average lunch price" class="c-rating-v3__time c-rating-v3__time--lunch" role="img"></i><span class="c-rating-v3__val">JPY 40,000～JPY 49,999</span>
                          </li>
```
Area
```
<div class="list-rst__area-genre cpy-area-genre">
                            Suitengumae Sta. / Sushi
                          </div>
```

Rating
```
<p class="c-rating c-rating--xxl c-rating--val45 list-rst__rating-total is-highlight cpy-total-score">
                            <i class="c-rating__star list-rst__rating-star"></i><span class="c-rating__val c-rating__val--strong list-rst__rating-val">4.66</span>
                          </p>
```

Rating Badge
```
<p class="c-rating-border c-rating-border--excellent">
                                        Excellent
                                      </p>
```


In [3]:
# # Set number of pages to get
# max_pages = 1
# page_count = 1

# df = pd.DataFrame(columns = ['Name',
#             'Rating',
#             'Lunch Price',
#             'Dinner Price',
#             'Station',
#             'Genre',
#             'Link',
#             'Address'])

# while page_count <= max_pages:

#     # Base information
#     print('On page ', page_count)
#     url = 'https://tabelog.com/en/tokyo/rstLst/' + str(page_count) +'/?SrtT=rt'
#     page = requests.get(url)
#     soup = BeautifulSoup(page.text,'html.parser')

#     # Extracting names, ratings, and areas from the soup object
#     names_list = soup.find_all(class_='list-rst__rst-name-target cpy-rst-name')
#     ratings_list = soup.find_all(class_='c-rating__val c-rating__val--strong list-rst__rating-val')
#     area_list = soup.find_all(class_="list-rst__area-genre cpy-area-genre")
#     soup_list = soup.find_all('div', class_='list-rst js-bookmark js-rst-cassette-wrap')

#     # Creating a DataFrame from the data dictionary


#     names = [0]*len(names_list)
#     ratings = [0]*len(names_list)
#     areas = [0]*len(names_list)
#     genres = [0]*len(names_list)
#     urls = [0]*len(names_list)
#     address = [0]*len(names_list)
#     lunch_price = [0]*len(names_list)
#     dinner_price = [0]*len(names_list)

#     # Creating a dictionary with the extracted data
#     data = {
#             'Name': names,
#             'Rating': ratings,
#             'Lunch Price': lunch_price,
#             'Dinner Price': dinner_price,
#             'Station': areas,
#             'Genre': genres,
#             'Link': urls,
#             'Address': address
#         }

#     # Extracting lunch and dinner prices, areas, genre of food
#     for i in range(len(soup_list)):
#         soup_tmp = soup_list[i].find_all('span', class_='c-rating-v3__val')
#         soup_tmp_area = soup_list[i].find('div', class_='list-rst__area-genre cpy-area-genre')

#         names[i] = (names_list[i].text)
#         ratings[i] = (ratings_list[i].text)

#         dinner_price[i] = (soup_tmp[0].text.strip())
#         lunch_price[i] = (soup_tmp[1].text.strip())

#         #print('original', soup_tmp)
#         soup_tmp_area = soup_tmp_area.get_text()
#         #print('text', soup_tmp)
#         if (type(soup_tmp_area) is list):
#             areas[i]  = soup_tmp_area.strip().split(' / ')[0].replace('\n', '')
#             genres[i] = soup_tmp_area.strip().split(' / ')[1].replace('\n', '')
#         else:
#             areas[i] = '-'
#             genres[i] = soup_tmp_area.strip().split(' / ')[0].replace('\n', '')

#         # Extracting the URL
#         urls[i] = soup_list[i].get('data-detail-url')


#         # # Getting URL information
#         # temp_page = requests.get(urls[i])
#         # url_soup = BeautifulSoup(temp_page.text,'html.parser')
#         # address[i] = url_soup.find(class_="rstinfo-table__address").text.strip()

#         temp_data = {
#         'Name': names,
#         'Rating': ratings,
#         'Lunch Price': lunch_price,
#         'Dinner Price': dinner_price,
#         'Station': areas,
#         'Genre': genres,
#         'Link': urls,
#         'Address': address
#         }

#         temp_df = pd.DataFrame(temp_data)
        
#     # Increase counter for pages
#     page_count += 1
#     df = pd.concat([df, temp_df])